In [8]:
findaccuracy(predictedvals,groundtruthvals) = sum(predictedvals.==groundtruthvals)/length(groundtruthvals)

findaccuracy (generic function with 1 method)

In [1]:
using RDatasets
using MLBase
using Plots
using Distances
using Random
using LinearAlgebra
using DataStructures
using NearestNeighbors

In [2]:
function perclass_splits(y,at)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, at) 
        push!(keepids,rowids...)
    end
    return keepids
end

perclass_splits (generic function with 1 method)

In [3]:
iris = dataset("datasets", "iris")
X = Matrix(iris[:,1:4])
irislabels = iris[:,5]
irislabelsmap = labelmap(irislabels)
y = labelencode(irislabelsmap, irislabels)
trainids = perclass_splits(y,0.7)
testids = setdiff(1:length(y),trainids)

40-element Vector{Int64}:
   8
   9
  12
  18
  23
  25
  32
  43
  45
  47
  49
  55
  59
   ⋮
 107
 111
 112
 116
 118
 119
 122
 125
 127
 134
 142
 149

In [4]:
Xtrain = X[trainids,:]
ytrain = y[trainids]
kdtree = KDTree(Xtrain')

KDTree{StaticArrays.SVector{4, Float64}, Euclidean, Float64}
  Number of points: 110
  Dimensions: 4
  Metric: Euclidean(0.0)
  Reordered: true

In [5]:
queries = X[testids,:]

40×4 Matrix{Float64}:
 5.0  3.4  1.5  0.2
 4.4  2.9  1.4  0.2
 4.8  3.4  1.6  0.2
 5.1  3.5  1.4  0.3
 4.6  3.6  1.0  0.2
 4.8  3.4  1.9  0.2
 5.4  3.4  1.5  0.4
 4.4  3.2  1.3  0.2
 5.1  3.8  1.9  0.4
 5.1  3.8  1.6  0.2
 5.3  3.7  1.5  0.2
 6.5  2.8  4.6  1.5
 6.6  2.9  4.6  1.3
 ⋮              
 4.9  2.5  4.5  1.7
 6.5  3.2  5.1  2.0
 6.4  2.7  5.3  1.9
 6.4  3.2  5.3  2.3
 7.7  3.8  6.7  2.2
 7.7  2.6  6.9  2.3
 5.6  2.8  4.9  2.0
 6.7  3.3  5.7  2.1
 6.2  2.8  4.8  1.8
 6.3  2.8  5.1  1.5
 6.9  3.1  5.1  2.3
 6.2  3.4  5.4  2.3

In [6]:
idxs, dists = knn(kdtree, queries', 5, true)

([[33, 39, 1, 21, 23], [32, 4, 11, 38, 37], [24, 21, 25, 7, 39], [1, 34, 5, 33, 23], [7, 3, 31, 34, 38], [24, 21, 25, 19, 33], [17, 22, 23, 30, 9], [32, 38, 4, 3, 7], [6, 18, 16, 36, 21], [16, 18, 5, 22, 9]  …  [109, 92, 108, 85, 89], [107, 109, 99, 103, 96], [95, 78, 81, 98, 88], [88, 78, 98, 79, 94], [104, 83, 110, 84, 101], [87, 105, 103, 82, 77], [89, 91, 101, 108, 66], [66, 56, 89, 91, 61], [107, 102, 82, 109, 103], [99, 109, 103, 100, 106]], [[0.09999999999999964, 0.14142135623730964, 0.17320508075688762, 0.22360679774997902, 0.22360679774997916], [0.14142135623730948, 0.29999999999999966, 0.34641016151377546, 0.36055512754639873, 0.42426406871192807], [0.22360679774997858, 0.2828427124746191, 0.2999999999999998, 0.3000000000000002, 0.30000000000000027], [0.09999999999999998, 0.14142135623730917, 0.17320508075688756, 0.1732050807568878, 0.17320508075688806], [0.45825756949558405, 0.5099019513592785, 0.5099019513592788, 0.5196152422706636, 0.5656854249492379], [0.3741657386773937,

In [9]:
c = ytrain[hcat(idxs...)]
possible_labels = map(i->counter(c[:,i]),1:size(c,2))
predictions_NN = map(i->parse(Int,string(string(argmax(possible_labels[i])))),1:size(c,2))
findaccuracy(predictions_NN,y[testids])

0.925